In [2]:
import pandas as pd

In [63]:
def make_arbStep(estep,filename,scale = pd.Series(1.2**(np.linspace(-4,4,9))),mode = 'HiTH'):
    volt_step = {'HiTH':pd.DataFrame({'U+':[61.91,119.41,229.42,451.48,870.61,1782.40,3452],
                            'U-':[30.55,58.92,113.19,222.75,429.54,879.40,1701]}),
                 'HiRes':pd.DataFrame({'U+':[40.30,77.73,149.34,293.89,566.72,1160.24,2248.26],
                            'U-':[13.65,26.33,50.59,99.56,191.98,393.03,761.60]})}
    
    things = volt_step[mode].loc[estep-1].apply(lambda x: x*scale)
#     return(things)
    maxN = 1701
    maxP = 3452
    
    #GSEOS lims
    # 1701.69
    # 3462.36
    close_neg = abs(things.T['U-']-maxN)<10
    things.loc['U-',close_neg] = maxN
    close_pos = abs(things.T['U+']-maxP)<10
    things.loc['U+',close_pos] = maxP
    
    locr = (things.T['U-']<=maxN)&(things.T['U+']<=maxP)
    things = things.T.loc[locr].T

    fup = lambda volt: 'send ILO_ESA_P_RMP(TARGET = %.2f,STEPS = 3447,DWELL = %d)'%(volt,0)
    fdn =lambda volt: 'send ILO_ESA_N_RMP(TARGET = %.2f,STEPS = 1699,DWELL = %d)'%(volt,0)

    cmds = things.apply(lambda x :'\n'.join([fup(x['U+']),fdn(x['U-'])]))
    
    cmds[0] = cmds[0]+'\n\nmsgbox info "wait here until ready"'

    
    with open(filename,'w') as f: 
        
        f.write('script %s() \nbegin\n#>>> from Gseos import PlaySound\nWaitTime = ask "Enter collection time in seconds"\n'%filename.replace('.gstol',''))
        cmds.apply(lambda x: f.write('\n%s \nwait timeout $WaitTime\n'%(x)))
        f.write(r'# >>> PlaySound("C:\IMAP\Instruments\Common\Sounds\Time to die.wav")')
        f.write('\nend')
        f.close()

In [64]:
estep_norm =pd.Series([0.0347,0.0670,0.1287,0.2533,0.4884,1.0000,1.935])

In [65]:
make_arbStep(6,'DER2_HiTh_Esteps.gstol',estep_norm,'HiTH')
make_arbStep(6,'DER2_HiRes_Esteps.gstol',estep_norm,'HiRes')

In [66]:
for i in range(1,8):
    stuff = make_arbStep(i,'DER1_HiTh_estep%d_v2.gstol'%i,scale = pd.Series(1.2**(np.arange(-4,8))),mode = 'HiTH')
    stuff = make_arbStep(i,'DER1_HiRes_estep%d_v2.gstol'%i,scale = pd.Series(1.1**(np.arange(-4,8))),mode = 'HiRes')

In [67]:
np.linspace(-4,4,9)

array([-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])